# **Python Web Mapping Using Folium.**

**Table of Contents.**

1. Introduction.
2. Install and import modules.
3. Data cleaning and exploration.
4. Folium Maps.

    4.1. A basic folium map.

    4.2. Loaction markers.

    4.3. Cirle Markers.

    4.4. GeoJson layer.

    4.5. Minimap.

    4.6. Search bar.

    4.7. Terminator.

    4.8. Routes.

    4.9. Ant Routes.

    4.10. Cluster Markers.

    4.11. Heatmaps.

    4.12. Choropleth Map.

    4.13. Coordinates Tool.

    4.14. Measure Tool.

    4.15. Dual Map.

5. References.

# 1. Introduction.

Interactive geographic maps provide a better view of hidden information in data. These maps allow users to interact through features such as : zooming in and out, selecting layers, panning and clickable data points. By stackin multiple layers geographic content, one can display massive complex data and share them widely as a website.
Using a Python's Folium library, we are able to access free and open source Leaflet JavaScript library through an API.
Leaflet js is used to dispaly simple, light weight and interactive maps online as layer such as : markers, popups, GeoJson, image overlays, WMS, and vector data types.
 We will build maps by exploring major Africa cities.

# 2. Install and import modules.

In [ ]:
# Install modules
!pip install mapclassify geopandas pandas folium

# Import modules
import geopandas as gpd            # Read geojson and shapefile
import pandas as pd                # Read csv file
import folium                      # Web maps
from mapclassify import classify    # web maps
from folium import plugins         # Extends folium functionality


# 3. Explore Data.

We need to load data into our workspace from Google Colab and perform data analysis. Since raw data contains uneccessary noise i.e. unwanted columns, we are going to clean it and finally convert the geojson countries file to geodataframe.

In [24]:
# Read datasets

# African countries in geojson format
ctry = gpd.read_file(r'/content/africa-countries.geojson')

# Worldcities in csv.
cities = pd.read_csv(r'/content/worldcities.csv')

Explore countries ppolygon data.

In [25]:

# View all columns of a dataframe
pd.set_option('display.max_columns', None)
print(ctry.columns)

Index(['scalerank', 'featurecla', 'labelrank', 'sovereignt', 'sov_a3',
       'adm0_dif', 'level', 'type', 'admin', 'adm0_a3', 'geou_dif', 'geounit',
       'gu_a3', 'su_dif', 'subunit', 'su_a3', 'brk_diff', 'name', 'name_long',
       'brk_a3', 'brk_name', 'brk_group', 'abbrev', 'postal', 'formal_en',
       'formal_fr', 'note_adm0', 'note_brk', 'name_sort', 'name_alt',
       'mapcolor7', 'mapcolor8', 'mapcolor9', 'mapcolor13', 'pop_est',
       'gdp_md_est', 'pop_year', 'lastcensus', 'gdp_year', 'economy',
       'income_grp', 'wikipedia', 'fips_10', 'iso_a2', 'iso_a3', 'iso_n3',
       'un_a3', 'wb_a2', 'wb_a3', 'woe_id', 'adm0_a3_is', 'adm0_a3_us',
       'adm0_a3_un', 'adm0_a3_wb', 'continent', 'region_un', 'subregion',
       'region_wb', 'name_len', 'long_len', 'abbrev_len', 'tiny', 'homepart',
       'filename', 'geometry'],
      dtype='object')


In [ ]:
# Subset columns from african countries data
africa = ctry[['admin', 'income_grp', 'iso_a3', 'pop_est','subregion', 'region_wb', 'geometry']]

# Rename iso_a3 column to iso3
africa.rename(columns = {'iso_a3':'iso3'}, inplace=True)

In [27]:
# Find if there is any duplicate values

any(africa.duplicated())

False

In [28]:
# View a subset of the cleaned dataframe, the first
africa.head(5)

,admin,income_grp,iso3,pop_est,subregion,region_wb,geometry
0,Burundi,5. Low income,BDI,8988091,Eastern Africa,Sub-Saharan Africa,"POLYGON ((29.34000 -4.49998, 29.27638 -3.29391..."
1,Benin,5. Low income,BEN,8791832,Western Africa,Sub-Saharan Africa,"POLYGON ((2.69170 6.25882, 1.86524 6.14216, 1...."
2,Burkina Faso,5. Low income,BFA,15746232,Western Africa,Sub-Saharan Africa,"POLYGON ((-2.82750 9.64246, -3.51190 9.90033, ..."
3,Ivory Coast,4. Lower middle income,CIV,20617068,Western Africa,Sub-Saharan Africa,"POLYGON ((-2.85613 4.99448, -3.31108 4.98430, ..."
4,Democratic Republic of the Congo,5. Low income,COD,68692542,Middle Africa,Sub-Saharan Africa,"POLYGON ((30.83386 3.50917, 30.77335 2.33988, ..."


In [19]:
# Create a simple web map of countries data

africa.explore()

Explore cities data.

In [29]:
# Data exploration world cities data

print(cities.columns)

Index(['city', 'city_ascii', 'lat', 'lng', 'country', 'iso2', 'iso3',
       'admin_name', 'capital', 'population', 'id'],
      dtype='object')


In [30]:
# Convert cities data to geodataframe

cities = gpd.GeoDataFrame(cities,                                                    # Data
                          geometry = gpd.points_from_xy(cities.lng, cities.lat),     # columns containing location infrormation
                          crs = 4326)

In [41]:
# Filter primary cities that are also found in our african polygon dataset.

african_cities_pop = cities[((cities['capital'] == 'primary') & (cities.iso3.isin(list(africa.iso3))))]


67

Subset data , filter only primary cities in Africa and select needed columns.

In [44]:

# Filter primary cities that are also found in our african polygon dataset.
african_cities_pop = cities[(cities['capital'] == 'primary') & (cities.iso3.isin(list(africa.iso3)))]

# Select important columns.
african_cities_pop = african_cities_pop[['city_ascii', 'lat', 'lng', 'country', 'iso3', 'population', 'geometry']]

# View data
african_cities_pop

67


,city_ascii,lat,lng,country,iso3,population,geometry
10,Cairo,30.0444,31.2358,Egypt,EGY,19787000.0,POINT (31.23580 30.04440)
19,Lagos,6.4500,3.4000,Nigeria,NGA,15487000.0,POINT (3.40000 6.45000)
22,Kinshasa,-4.3317,15.3139,Congo (Kinshasa),COD,15056000.0,POINT (15.31390 -4.33170)
47,Luanda,-8.8383,13.2344,Angola,AGO,8883000.0,POINT (13.23440 -8.83830)
68,Dar es Salaam,-6.8000,39.2833,Tanzania,TZA,7461000.0,POINT (39.28330 -6.80000)
...,...,...,...,...,...,...,...
783,Mbabane,-26.3208,31.1617,Swaziland,SWZ,94874.0,POINT (31.16170 -26.32080)
795,Gitega,-3.4283,29.9250,Burundi,BDI,41944.0,POINT (29.92500 -3.42830)
801,Banjul,13.4531,-16.5775,The Gambia,GMB,31356.0,POINT (-16.57750 13.45310)
824,Lobamba,-26.4465,31.2064,Swaziland,SWZ,5800.0,POINT (31.20640 -26.44650)


# 4. Folium maps.

**4.1 A basic folium map.**

Lets create a basic folium map with

In [81]:
# Create folium map with different basemaps

# Make an empty map
map = folium.Map(location=[5, 24],     # Approximate central location for africa
                 tiles="Stamen Watercolor",  # Add base map
               zoom_start=3,         # Low zoom level less detail, zoomed in.
               width=900,             # Map Pixel size horizontally
               height=600,            # Map Pixel size vertically
               control_scale = True   # Adds distance scale
              )

# add different tiles/basemaps to map which users can select or unselect any layer
folium.raster_layers.TileLayer('CartoDB Positron').add_to(map)
folium.raster_layers.TileLayer('Open Street Map').add_to(map)
folium.raster_layers.TileLayer('CartoDB Dark_Matter').add_to(map)
folium.raster_layers.TileLayer('Stamen Terrain').add_to(map)
folium.raster_layers.TileLayer('Stamen Toner').add_to(map)

# add layer control , allow toggling of layers.
folium.LayerControl().add_to(map)

# Show map
map

**4.2. Loaction markers.**

Each marker has a pop up that when clicked will display city name and its corresponding population.

In [49]:
# Add locations marker to major african cities with each pop up showing city and its population.

for index, location_info in african_cities_pop .iterrows():

    # Add marker to each location
    folium.Marker([location_info.lat, location_info.lng],
                  # Pop up shows city and population message when clicked.
                  popup=location_info.city_ascii +' ' + str(location_info.population)).add_to(map)
map

Add a different type of marker.

More icons include ['bicycle', 'car', 'bus', 'truck', 'motorcycle']

In [51]:
# Lets add different types of markers for population

for index, location_info in african_cities_pop .iterrows():

    # Population marker
    folium.Marker([location_info.lat, location_info.lng],
                  popup=location_info.population,          # Clicking the icon will show population for that city
                  icon = folium.Icon(icon = "info-sign")).add_to(map)
# Call map
map

Enumerate markers using a Carto DB base map style. Loop using itertools.

In [83]:
# Initiate an enumerated map
enum = folium.Map(location=[5, 24], tiles="CartoDB Positron",
                  zoom_start=3, width=900, height=600,control_scale = True)

for index, location_info in african_cities_pop.iterrows():

    # Population marker
    folium.Marker([location_info.lat, location_info.lng],
                  # popup=location_info.population,
                  icon = plugins.BeautifyIcon(number = location_info.city_ascii,
                                          border_color = "green",
                                          border_width = "2",
                                          background_color = 'yellow',  # Others : "Transparent"
                                          text_color = "purple",
                                            # Try inner_icon_style='margin-top:0px;'
                                          inner_icon_style='font-size:10px;margin-top:1px')
                 ).add_to(enum)

folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(enum)

enum


Enumerate markers again but loop using itertuples.

In [62]:
# Initiate an enumerated map
enum = folium.Map(location=[5, 24],
                  tiles=None,   # No basemap
                  zoom_start=3, width=900, height=600,control_scale = True)

# Use itertuple to loop through
for i in african_cities_pop.itertuples():
    folium.Marker([i.lat, i.lng],  # Location
                 popup = i.population,
                 icon=plugins.BeautifyIcon(number = i.iso3,
                                          border_color = "green",
                                          border_width = "1",
                                          text_color = "red",
                                          inner_icon_style='margin-top:0px;')).add_to(enum)

# Call map
enum

**4.3.  Cirle Markers.**

Add a customized circle marker to all the cities. The circle will always maintain the same size irrespective of zoom level.

In [87]:
# Initiate a second map
map = folium.Map(location=[5, 24], tiles='Stamen Terrain',zoom_start=3, width=900, height=600,control_scale = True)

# add different tiles to map
folium.raster_layers.TileLayer('CartoDB Dark_Matter').add_to(map)
folium.raster_layers.TileLayer('CartoDB Positron').add_to(map)
folium.raster_layers.TileLayer('Open Street Map').add_to(map)

# Loop through each city and add a circle marker.
for index, location_info in african_cities_pop .iterrows():

    # city marker
    folium.CircleMarker([location_info.lat, location_info.lng],
                        radius=10,                                # Pixel value
                        fill=True,
                        color='green',                            # Edge colour
                        fill_color='red',                         # Circle fill colour
                        fill_opacity=0.25 ).add_to(map)

# add layer control to show different maps
folium.LayerControl().add_to(map)
# Show map
map


Add circle whose sizes are relative to the population, size corresponding to population.

In [63]:
# Initiate a second map
pop = folium.Map(location=[5, 24], tiles="Stamen Toner" ,zoom_start=3, width=900, height=600,control_scale = True)

# add different tiles to map
folium.raster_layers.TileLayer('CartoDB Dark_Matter').add_to(pop)
folium.raster_layers.TileLayer('Stamen Toner').add_to(pop)
folium.raster_layers.TileLayer('CartoDB Positron').add_to(pop)
folium.raster_layers.TileLayer('Open Street Map').add_to(pop)

# Loop through each city and add a circle marker.
for index, location_info in african_cities_pop.iterrows():

    # city marker
    folium.Circle([location_info.lat, location_info.lng],
                        radius=location_info.population/30,                                # Raduis in meters.
                        fill=True,
                        color='green',
                        fill_color='red',
                        fill_opacity=0.25 ).add_to(pop)

# add layer control to show different maps
folium.LayerControl().add_to(pop)

# Show map
pop

**4.4. GeoJson layer.**

Overlay multiple layers including : countries geojson, population circle marker and city layer.

In [64]:
# Add multiple layers , countries layer, population layer , and city layer.

map = folium.Map(location=[5, 24], zoom_start=3, width=900, height=600,control_scale = True)

# Add countries geojson layer
folium.GeoJson(ctry, name="African Countries").add_to(map)


# Add basemaps
folium.raster_layers.TileLayer('CartoDB Positron').add_to(map)
folium.raster_layers.TileLayer('Open Street Map').add_to(map)


# Add population layer
# Loop through each city and add a circle marker.
for index, location_info in african_cities_pop.iterrows():

    # city population marker
    folium.Circle([location_info.lat, location_info.lng],
                        radius=location_info.population/30,                # Radius in meters.
                        popup=location_info.population,
                        fill=True,
                        color='green',
                        fill_color='red',
                                          fill_opacity=0.25).add_to(map)


# Add cities layer
# Loop through each city and add a circle marker.
for index, location_info in african_cities_pop .iterrows():

    # city marker
    folium.CircleMarker([location_info.lat, location_info.lng],
                        popup=location_info.city_ascii,
                        radius=10,                                # Pixel value
                        fill=True,
                        color='green',                            # Edge colour
                        fill_color='red',                         # Circle fill colour
                        fill_opacity=0.25).add_to(map)


# Add layer control
folium.LayerControl().add_to(map)

map

**4.5. Minimap.**

A minimalistic map than full scale map that removes objects and details but show essential information. The show location.

In [65]:
# Add a minimap to cities data at the bottom of the main map.

# Initiate a second map
map = folium.Map(location=[5, 24], zoom_start=3, width=900, height=600,control_scale = True)

# Loop through each city and add a circle marker.
for index, location_info in african_cities_pop.iterrows():

    # city marker
    folium.Marker([location_info.lat, location_info.lng],
                        popup=location_info.city_ascii,
                        icon=folium.Icon(color='green', icon = "info-sign")).add_to(map)

# Add mini map
plugins.MiniMap().add_to(map)

# Add scroll zoom toggler to map
plugins.ScrollZoomToggler().add_to(map)

# Add full screen button to map
plugins.Fullscreen(position='topright').add_to(map)

# Add layer control to show different maps
folium.LayerControl().add_to(map)

# Show map
map

**4.6. Search bar.**

Creates a map with search box located inside the map and allows user to type and identify names of places across the globe. The map automatically zooms to that location when clicked.

In [66]:
# Initiate a second map
search = folium.Map(location=[5, 24], zoom_start=3, width=900, height=600,control_scale = True)

# Add plugin
plugins.Geocoder().add_to(search)

# Call map
search

**4.7. Terminator.**

Terminator shows the separation between the day and the night at the exact moment the map was created.

In [67]:
# Initiate a second map
term = folium.Map(location=[5, 24], zoom_start=3, width=900, height=600,control_scale = True)

# Add plugin
plugins.Terminator().add_to(search)

# Call map
term

**4.8. Routes.**

Using a line lets connect seven most popular cities on the map.

In [69]:
# Filter the most populous cities , more than 5m people
largest = african_cities_pop[african_cities_pop.population > 5000000].head(6)

# Extract geometries x, y coordinates
coord = list(zip(largest.lat, largest.lng))

# Initialise map
route = folium.Map([5, 24], zoom_start=3, tiles ='CartoDB Positron',control_scale = True)

# Add basemap
folium.raster_layers.TileLayer('CartoDB Positron').add_to(route)

# Add route to search map
folium.PolyLine(coord).add_to(route)

# Add layer control
folium.LayerControl().add_to(route)

# Call map
route

**4.9. Ant Routes.**


In [70]:
# Add ant paths route to our large cities

# Initialise map
ant_path = folium.Map([5, 24], tiles = 'CartoDB Positron', zoom_start=4)

# Add path
plugins.AntPath(coord).add_to(ant_path)

ant_path

**4.10. Cluster Markers.**

Cluster markers group toether multiple markers based on zoom level.

In [73]:
 # Get x and y coordinate for each point and store them in new columns X and Y.
african_cities_pop['X'] = african_cities_pop['geometry'].apply(lambda geom: geom.x)
african_cities_pop['Y'] = african_cities_pop['geometry'].apply(lambda geom: geom.y)

# Create a list of coordinate pairs,
locations = list(zip(african_cities_pop['Y'], african_cities_pop['X']))

# initialise map
cluster = folium.Map(location=[5, 24], zoom_start=3, width=900, height=600,control_scale = True)

# Add cluster
plugins.MarkerCluster(locations).add_to(cluster)
cluster

**4.11. Heatmaps.**

They show georaphical clustering of a certaing phenomenon by using different levels of concentration of an entity.

In [74]:
# A heatmap of african cities location

# Make an empty map
heat = folium.Map(location=[5, 24], zoom_start=3, width=900, height=600,control_scale = True)

# Add marker to each location
plugins.HeatMap(locations).add_to(heat)


# Call map
heat

**4.12. Choropleth Map.**

Lets plot choropleth map

In [76]:
# First let do some data scrubbing , enhances accuracy, consistency , and reliability.

# Create a unique column for each feature in africa data, this will be required
# by folium as a unique identifier for each feature
africa['geo_id'] = africa.index.astype(str)

# Select needed column
africa_subset = africa[['geo_id', 'pop_est', 'geometry']]
africa_subset.head()

# Convert to geo-json file
africa_pop = africa_subset.to_json()

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [77]:
# Plot choropleth with a mouse position cursor showing coordinates at the top left corner.

# Create map instance
choro = folium.Map([5, 24], tiles = 'CartoDB Positron', zoom_start = 3)

# Interactive choropleth map
folium.Choropleth(geo_data = africa_pop, # A geo_json  file
                 name = "African Countries Population",
                 data = africa_subset,  # A geodataframe
                 columns = ['geo_id', 'pop_est'],
                 key_on = 'feature.id',
                  #bins = 5,                # Number of equal bins
                  bins = [-100, 10000000, 40000000, 60000000, 100000000, 200000000],
                 fill_color = 'YlOrRd',     # Colour scheme "BuPu",
                 fill_opacity = 0.5,        # Polygon transparency ratio
                 line_opacity = 0.5,        # Line transparency
                 line_color='black',     # Borders colour
                 legend_name = 'Population',
                  smooth_factor = 1.0,
                 highlight = True        # Will highlihgt polygon when hovering
                 ).add_to(choro)

# Add mouse position, format to two decimal places
formatter = "function(num) {return L.Util.formatNum(num, 2) + ' º ';};"

plugins.MousePosition(
    position="topleft",
    separator="  ,  ",
    empty_string="NaN",
    lng_first=True,
    num_digits=20,
    prefix="Coordinates:",
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(choro)

# layer control to turn choropleth on or off
folium.LayerControl().add_to(choro)

# Call map
choro

**4.13. Coordinates Tool.**

This feature allows users to click any position on the map and the map will display latitude & longitude information.**bold text**

In [78]:
# Initilise map
map_lat_long = folium.Map(location=[5, 24], zoom_start=4)

# add latitude and longitude tool to map
map_lat_long.add_child(folium.LatLngPopup())

# display map
map_lat_long

**4.14. Measure Tool.**

Two points will be selected on the map and the distance between them output on the map canvas.

In [79]:
# Initialise map
measure_dist = folium.Map([5, 24], zoom_start=4)

# measure control
measure_control = plugins.MeasureControl(position='topright',
                                         active_color='blue',
                                         completed_color='red',
                                         primary_length_unit='kilometers')

# add measure control to map
measure_dist .add_child(measure_control)

# Call map to display
measure_dist

**4.15. Dual Map.**

In [80]:
# dual map
map_dual = plugins.DualMap(location=[5, 24], tiles=None, zoom_start=4)

# map tiles
folium.TileLayer('Stamen Terrain').add_to(map_dual)
folium.TileLayer('CartoDB Positron').add_to(map_dual)

# add layer control to maps
folium.LayerControl().add_to(map_dual)

# display map(s)
map_dual

# 5. References.

1. African Countries geojson - https://cartographyvectors.com/map/6-africa-outline-with-countries
2. World Cities Population data - https://simplemaps.com/data/world-cities
